In [ ]:
# !featurize dataset download 6878dbe0-2e3d-4065-92e0-1db6ec358071

In [ ]:
# pip3 install --user torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113 --force-reinstall --upgrade

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.models import resnet50, convnext_tiny, convnext_base


import utils
import models
import label_processor

In [ ]:
DATA_PATH = '/home/featurize/data/medium/'
# DATA_PATH = './dataset/medium/'
embed = label_processor.CStdLib(single=True)
CLASS_NUM = embed.class_num

dataset = utils.ColorfulClothesCLF(DATA_PATH, class_num=CLASS_NUM, embed=embed, train=True)
data_distribution = dataset.clean_and_analyse()
print(data_distribution)
small_len = int(len(dataset) * 0.002)
small, _ = random_split(dataset, [small_len, len(dataset) - small_len])
val_len = int(len(small) * 0.1)
train, val = random_split(small, [len(small) - val_len, val_len])

train_loader = DataLoader(train, batch_size=16, shuffle=True, num_workers=4)
test_loader = DataLoader(val, batch_size=1, shuffle=False, num_workers=4)

In [ ]:
model = convnext_tiny(pretrained=True)
num_params = sum(p.numel() for p in model.parameters())
print("Number of parameters: {}".format(num_params))
print(model)

In [ ]:
model.fc = nn.Linear(768, CLASS_NUM)

In [ ]:
loss_weight = (1. / data_distribution).to(torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'))
criterion = nn.CrossEntropyLoss(reduction='mean', weight=loss_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
trainer = utils.Trainer(model, class_num=CLASS_NUM, criterion=criterion, optimizer=optimizer, milestones=[5, 10, 15])

In [ ]:
trainer.train(train_loader=train_loader, test_loader=test_loader, epochs=20)